# Day 20
https://adventofcode.com/2020/day/20

In [1]:
import aocd
data = aocd.get_data(year=2020, day=20)

In [2]:
from collections import Counter, deque
from math import sqrt
from itertools import combinations
import numpy as np
import regex as re

##### Part 1: Find the corners

In [3]:
re_title = re.compile(r'Tile (\d+)')
re_row = re.compile(r'([#\.]+)')
def read_tiles(text):
    tiles = {}
    for tiletext in text.split('\n\n'):
        titlematch = re_title.search(tiletext)
        rowmatches = re_row.findall(tiletext)
        if titlematch and rowmatches:
            number = int(titlematch.group(1))
            tile = np.array([[1 if char == '#' else 0 for char in row] for row in rowmatches])
            tiles[number] = tile
    return tiles

In [4]:
def edge_matches(edge, other_tile):
    for other_edge in (other_tile[0], other_tile[:, -1], other_tile[:, 0], other_tile[-1]):
        if (edge == other_edge).all():
            return True
        if (np.flip(edge) == other_edge).all():
            return True
    return False

In [5]:
def edges_match(first, second):
    return any(edge_matches(edge, second) for edge in (first[0], first[:, -1], first[:, 0], first[-1]))

In [6]:
def all_matching_edges(tiles):
    return {(first, second) for first, second in combinations(tiles.keys(), 2)
            if edges_match(tiles[first], tiles[second])}

In [7]:
def find_correct_layout(tiles, matching_edges):
    grid_size = int(sqrt(len(tiles)))
    tile_neighbours = Counter([first for first, second in matching_edges]
                              + [second for first, second in matching_edges])
    first_corner = next(tile for tile, qty in tile_neighbours.items() if qty == 2)
    
    search = deque()
    search.append([first_corner])
    
    while search:
        placed = search.pop()
        position = len(placed)
        
        if position == len(tiles):
            return placed
        
        above = position - grid_size if position >= grid_size else None
        left = position - 1 if position % grid_size > 0 else None
        right = position + 1 if position % grid_size < (grid_size - 1) else None
        below = position + grid_size if position + grid_size < len(tiles) else None
        neighbours = sum(1 for adj in (above, left, right, below) if adj is not None)
        
        not_yet_placed = {tile for tile in tiles.keys()
                          if tile not in placed and tile_neighbours[tile] == neighbours}
        
        for tile in not_yet_placed:
            placeable = True
            if above and not ((tile, placed[above]) in matching_edges or (placed[above], tile) in matching_edges):
                placeable = False
            if left and not ((tile, placed[left]) in matching_edges or (placed[left], tile) in matching_edges):
                placeable = False
            if placeable:
                search.append(placed + [tile])

In [8]:
tiles = read_tiles(data)
matching_edges = all_matching_edges(tiles)
layout = find_correct_layout(tiles, matching_edges)
p1 = layout[0] * layout[11] * layout[132] * layout[143]
print('Part 1: {}'.format(p1))

Part 1: 15003787688423


##### Part 2: Make the image and find the sea monsters

In [9]:
def tile_fits(tile, above, below, left, right):
    return all((
        above is None or edge_matches(tile[0], above),
        below is None or edge_matches(tile[-1], below),
        left is None or edge_matches(tile[:,0], left),
        right is None or edge_matches(tile[:,-1], right)
    ))

In [10]:
def transformed_to_fit(tile, above, below, left, right):
    for rotations in range(4):
        for hflip in (False, True):
            for vflip in (False, True):
                transformed = np.rot90(tile, rotations)
                if hflip:
                    transformed = np.fliplr(transformed)
                if vflip:
                    transformed = np.flipud(transformed)
                if tile_fits(transformed, above, below, left, right):
                    return transformed

In [11]:
def make_image(tiles, matching_edges, layout):
    grid_size = int(sqrt(len(layout)))
    parts = [tiles[tile] for tile in layout]
    
    # orient tiles to match their required neighbours in the layout
    for y in range(grid_size):
        for x in range(grid_size):
            current = (y * grid_size) + x
            above = parts[current - grid_size] if current >= grid_size else None
            below = parts[current + grid_size] if current + grid_size < len(layout) else None
            left = parts[current - 1] if current % grid_size > 0 else None
            right = parts[current + 1] if (current + 1) % grid_size != 0 else None
            parts[current] = transformed_to_fit(parts[current], above, below, left, right)
    
    # trim the outer elements from each tile
    parts = [tile[1:-1,1:-1] for tile in parts]
    
    # combine arrays into a single large array
    return np.concatenate([np.concatenate(parts[y*grid_size:(y+1)*grid_size], axis=1) for y in range(grid_size)])

In [12]:
sea_monster = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    [1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1],
    [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
])
def count_sea_monsters(image):
    monsters = 0
    rows, cols = image.shape
    monsterheight, monsterwidth = sea_monster.shape
    
    for rotations in range(3):
        for fliph in (False, True):
            for flipv in (False, True):
                adjusted_image = np.rot90(image, rotations)
                if fliph:
                    adjusted_image = np.fliplr(adjusted_image)
                if flipv:
                    adjusted_image = np.flipud(adjusted_image)

                for y in range(rows-monsterheight):
                    for x in range(cols-monsterwidth):
                        window = adjusted_image[y:y+monsterheight,x:x+monsterwidth]
                        if np.greater_equal(window, sea_monster).all():
                            monsters += 1
    
    return monsters

In [13]:
def sea_roughness(image):
    monsters = count_sea_monsters(image)
    return image.sum() - (sea_monster.sum() * monsters)

In [14]:
image = make_image(tiles, matching_edges, layout)
p2 = sea_roughness(image)
print('Part 2: {}'.format(p2))

Part 2: 1705
